In [17]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

import pandas as pd
import geopandas as gpd
from siuba import *

# Finding Transfer Points from Conveyal Paths

In [43]:
df = pd.read_csv('./Conveyal_Paths_Test.csv')
df.index.rename('rider_trip_id', inplace=True)
df = df.dropna() #  remove same o/d
df >> head(10)

,origin,destination,routes,boardStops,alightStops,rideTimes,accessTime,egressTime,transferTime,waitTimes,totalTime,nIterations
rider_trip_id,,,,,,,,,,,,
1,Archives,Curtis Park,533,7015,7019,9.0,1.1,17.6,0.0,1.9,29.6,1200
2,Archives,Winco,533|013,7034|1096,7043|3089,20.0|21.0,0.3,1.4,2.5,1.7|4.5,51.4,410
3,Archives,Winco,533|023,7034|1096,7043|1464,20.0|22.0,0.3,6.2,2.5,1.7|4.5,57.2,20
4,Archives,Winco,507|026|082,7015|316|1666,7055|1551|3089,16.0|12.0|15.0,1.1,1.4,4.7,1.9|2.7|1.5,56.4,390
5,Archives,Winco,533|023,7034|1096,7043|1464,20.0|21.0,0.3,6.2,2.5,1.7|4.5,56.2,40
6,Archives,Winco,533|084,7034|9801,7049|1074,32.0|10.0,0.3,3.4,1.0,1.7|12.0,60.4,340
7,Archives,Middle School,533|067,7015|2592,7022|2594,16.0|1.0,1.1,8.5,15.9,1.9|4.1,48.5,150
8,Archives,Middle School,533|067,7015|2592,7022|2594,16.0|2.0,1.1,8.5,15.9,1.9|5.1,50.5,450
9,Archives,Middle School,533,7015,7022,16.0,1.1,32.1,0.0,1.9,51.1,600


In [30]:
list(range(3))

[0, 1, 2]

In [20]:
df.columns

Index(['origin', 'destination', 'routes', 'boardStops', 'alightStops',
       'rideTimes', 'accessTime', 'egressTime', 'transferTime', 'waitTimes',
       'totalTime', 'nIterations'],
      dtype='object')

In [21]:
array_cols = ['routes', 'boardStops', 'alightStops',
       'rideTimes', 'waitTimes']

In [22]:
for col in array_cols:
    df.loc[:,col] = df[col].map(lambda x: x.split('|'))

In [38]:
df['xfer_event'] = df.boardStops.map(lambda x: list(range(len(x))))
## add information on which xfer the event represents, with 0 being the rider trip origin (not an xfer)

In [41]:
## not quite, need to drop first stop!
xfer_events = df.explode(array_cols + ['xfer_event'])

In [42]:
xfer_events

,origin,destination,routes,boardStops,alightStops,rideTimes,accessTime,egressTime,transferTime,waitTimes,totalTime,nIterations,xfer_event
rider_trip_id,,,,,,,,,,,,,
1,Archives,Curtis Park,533,7015,7019,9.0,1.1,17.6,0.0,1.9,29.6,1200,0
2,Archives,Winco,533,7034,7043,20.0,0.3,1.4,2.5,1.7,51.4,410,0
2,Archives,Winco,013,1096,3089,21.0,0.3,1.4,2.5,4.5,51.4,410,1
3,Archives,Winco,533,7034,7043,20.0,0.3,6.2,2.5,1.7,57.2,20,0
3,Archives,Winco,023,1096,1464,22.0,0.3,6.2,2.5,4.5,57.2,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Airport,Sunrise Mall,001,9801,2807,33.0,0.2,7.8,3.7,4.0,96.8,190,2
218,Airport,Sunrise Mall,db09a072-c976-498b-8824-becced337484,641945e4-f72f-4a09-9110-35d862f24266,b3ee0b92-804a-46b7-932b-e924dcb3ca7c,14.6,0.4,3.7,9.1,1.6,95.7,250,0
218,Airport,Sunrise Mall,Shuttle,SAC,RSV,30.0,0.4,3.7,9.1,3.7,95.7,250,1


In [46]:
xfer_points = xfer_events >> filter(_.xfer_event > 0) >> count(_.boardStops) >> arrange (-_.n)

In [47]:
xfer_points

,boardStops,n
45,7069,18
41,7051,13
51,9801,13
17,290,12
20,319,12
21,323,12
2,1099,11
35,7000,11
50,80337645-0a8d-44f2-b37f-5ecf8209b17f,10
55,9f772657-d3b2-4fc6-8080-e9e986a01eb9,10


## Test with SacRT Feed...